In [1]:
import pandas as pd
import json
import ast
import numpy as np

In [89]:
from pyzotero import zotero
zot = zotero.Zotero(1234567, 'user', 'asdfasdfasdfasdfasdfasdf')
items = zot.top(limit=5)
# we've retrieved the latest five top-level items in our library
# we can print each item's item type and ID
for item in items:
    print('Item Type: %s | Key: %s' % (item['data']['itemType'], item['data']['key']))

In [90]:
items[0]['data'].keys()

IndexError: list index out of range

In [91]:
pd.read_json('zot_crossref_processed.json')['ShortNames']

0                        [AIRX3STD, MAT1NXSLV, MAT1NXFLX]
1       [AIRX3STD, OMAEROe, NLDAS_NOAH0125_H, OMDOAO3e...
2                                              [AIRX3STD]
3                                              [AIRX3STD]
4                                              [AIRX3STD]
                              ...                        
4670                                          [TRMM_2B31]
4671                                          [TRMM_2B31]
4672                                          [TRMM_2B31]
4673                                          [TRMM_2B31]
4674                                          [TRMM_2B31]
Name: ShortNames, Length: 4675, dtype: object

In [92]:
'''
idf = pd.read_csv("zot2.csv")

item_type = []
good_index_list = []
for index, row in idf.iterrows():
    if type(row['itemType']) == float:
        if type(row['item_type']) != float and row['item_type'] != 'None':
            good_index_list.append(index)
            item_type.append(row['item_type'])

df = idf.iloc[good_index_list]
df['itemType'] = item_type
df.columns
'''

'\nidf = pd.read_csv("zot2.csv")\n\nitem_type = []\ngood_index_list = []\nfor index, row in idf.iterrows():\n    if type(row[\'itemType\']) == float:\n        if type(row[\'item_type\']) != float and row[\'item_type\'] != \'None\':\n            good_index_list.append(index)\n            item_type.append(row[\'item_type\'])\n\ndf = idf.iloc[good_index_list]\ndf[\'itemType\'] = item_type\ndf.columns\n'

In [93]:
df = pd.read_csv("zot2.csv")

item_type = []
dois = []
extras = []
for index, row in df.iterrows():
    if type(row['itemType']) == float:
        item_type.append('journalArticle')
    else:
        item_type.append(row['itemType'])
    
    if row['itemType'] != 'journalArticle' or row['itemType'] != 'conferencePaper':
        dois.append('')
        extras.append('DOI: ' + str(row['DOI']))
    else:
        dois.append(row['DOI'])
        extras.append(row['extra'])

df['itemType'] = item_type
df['DOI'] = dois
df['extra'] = extras
df['ShortNames'] = df['ShortNames'].apply(lambda x: [] if type(x) == float else ast.literal_eval(x))

In [94]:
zdf = df[['key', 'version', 'itemType', 'title', 'creators', 'abstractNote', 'publicationTitle', 'volume', 
          'issue', 'pages', 'date', 'series', 'journalAbbreviation', 'language', 'DOI', 'ISSN', 'shortTitle', 
          'url', 'accessDate', 'archiveLocation', 'libraryCatalog', 'callNumber', 'rights', 'extra', 'tags']]
        
#zdf = df[['key', 'version', 'itemType', 'title', 'creators', 'abstractNote', 'publicationTitle']]
        
zdf['creators'] = zdf['creators'].apply(lambda x: [] if type(x) == float else ast.literal_eval(x))
zdf['tags'] = zdf['tags'].apply(lambda x: [] if type(x) == float else ast.literal_eval(x))
zdf = zdf.replace(np.nan, '', regex=True)
#zdf['creators'] = zdf['creators'].apply(lambda x: ast.literal_eval(x) if type(x) != float else x)
#zdf['tags'] = zdf['tags'].apply(lambda x: ast.literal_eval(x) if type(x) != float else x)

/var/folders/pk/zld5ctsn7r9cvw4wpxfg6g8w0000gn/T/ipykernel_44891/2255984763.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zdf['creators'] = zdf['creators'].apply(lambda x: [] if type(x) == float else ast.literal_eval(x))
/var/folders/pk/zld5ctsn7r9cvw4wpxfg6g8w0000gn/T/ipykernel_44891/2255984763.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zdf['tags'] = zdf['tags'].apply(lambda x: [] if type(x) == float else ast.literal_eval(x))


In [95]:
for index, row in zdf.iterrows():
    for tag in df['ShortNames'][index]:
        zdf['tags'][index].append({'tag': tag, 'type': 2})

In [87]:
for index, row in zdf.iloc[0:3].iterrows():
    if index % 100 == 0:
        print("Index: " + str(index))
        
    asd = zot.create_items([row.to_dict()])
    
    if asd['failed']:
        print(index)

Index: 0


In [102]:
start = -1

if start == -1:
    start = 0
    endzone = len(zdf)
else:
    endzone = start + 1
    
for index, row in zdf.iloc[start:endzone].iterrows():
    if index % 100 == 0:
        print("Index: " + str(index))
    asd = zot.create_items([row.to_dict()])
    if asd['failed']:
        print(index)

Index: 0
Index: 100
Index: 200
Index: 300
Index: 400
Index: 500
Index: 600
Index: 700
Index: 800
Index: 900
Index: 1000
Index: 1100
Index: 1200
Index: 1300
Index: 1400
Index: 1500
Index: 1600
Index: 1700
Index: 1800
Index: 1900
Index: 2000
Index: 2100
Index: 2200
Index: 2300
Index: 2400
Index: 2500
Index: 2600
Index: 2700
Index: 2800
Index: 2900
Index: 3000
Index: 3100
Index: 3200
Index: 3300
Index: 3400
Index: 3500
Index: 3600
Index: 3700
3763
Index: 3800
Index: 3900
3933
Index: 4000
4085
Index: 4100
4102
4140
4171
4183
Index: 4200
4229
4252
4268
4270
4282


In [103]:
asd

{'successful': {'0': {'key': 'NJHTQBAQ',
   'version': 4286,
   'library': {'type': 'group',
    'id': 4736929,
    'name': 'tagged_zot',
    'links': {'alternate': {'href': 'https://www.zotero.org/groups/4736929',
      'type': 'text/html'}}},
   'links': {'self': {'href': 'https://api.zotero.org/groups/4736929/items/NJHTQBAQ',
     'type': 'application/json'},
    'alternate': {'href': 'https://www.zotero.org/groups/4736929/items/NJHTQBAQ',
     'type': 'text/html'}},
   'meta': {'createdByUser': {'id': 9632937,
     'username': 'kevingao26',
     'name': '',
     'links': {'alternate': {'href': 'https://www.zotero.org/kevingao26',
       'type': 'text/html'}}},
    'creatorSummary': 'Li',
    'parsedDate': '2022-01-01',
    'numChildren': 0},
   'data': {'key': 'NJHTQBAQ',
    'version': 4286,
    'itemType': 'journalArticle',
    'title': 'Long-term Variation of Absorption and Total Aerosol Optical Properties over Typical Provinces of China from Satellite Observations',
    'creato

In [10]:
template = zot.item_template('book')
template['creators'][0]['firstName'] = 'Monty'
template['creators'][0]['lastName'] = 'Cantsin'
template['title'] = 'Maris Kundzins: A Life'
resp = zot.create_items([template])